In [1]:
import pandas as pd

In [2]:
tweets = pd.read_csv('data/player_tweets.csv')
tweets.head(3)

,handle,time,text,mentions
0,JohnWall,2021-02-21 13:06:10,👌🏾‼️ https://t.co/Ptf5UmFPGp,NaN
1,JohnWall,2021-02-19 18:54:46,Thank you #LostAndFoundMidtown! More meals for...,NaN
2,JohnWall,2021-02-19 04:10:47,RT @SEMelbPhoenix: “We really missed the fans ...,SEMelbPhoenix


In [3]:
# remove all the tweets where the 'time' column makes no sense (i.e. NaN)
#   there are only 10 of these tweets
def is_time(t):
    return len(str(t)) == 19
set([len(str(t)) for t in tweets['time']])
tweets = tweets[tweets['time'].map(is_time)]
tweets['datetime'] = pd.to_datetime(tweets['time'])
def get_season(datetime):
    season = str(datetime.year)
    if datetime.month <= 6:
        season = str(datetime.year-1)+'-'+season
    else:
        season += '-'+str(datetime.year+1)
    return season
tweets['season'] = [get_season(x) for x in tweets['datetime']]
tweets.sample(5)

,handle,time,text,mentions,datetime,season
491145,Original_Turner,2018-01-07 05:31:49,RT @8pts9secs: T.J. Leaf did this. \n https://...,8pts9secs,2018-01-07 05:31:49,2017-2018
55695,greivisvasquez,2019-04-18 17:51:27,"Importante reunión con NBA Academy en Mexico, ...",NaN,2019-04-18 17:51:27,2018-2019
728767,wholeteamDot,2019-05-06 18:58:54,RT @Goofy757__: @wholeteamDot HAPPY BDAY TO TH...,"Goofy757__,wholeteamDot",2019-05-06 18:58:54,2018-2019
716304,YoungIvee,2019-04-29 20:19:57,RT @freeblackgirl: John Singleton is the first...,freeblackgirl,2019-04-29 20:19:57,2018-2019
788174,rob_williamsIII,2015-12-19 21:59:58,"She ain't ready for it, if I ain't the greates...",NaN,2015-12-19 21:59:58,2015-2016


In [7]:
grouped_freq = tweets.groupby(['handle', 'season']).count()
#grouped_multiple.columns = ['age_mean', 'age_min', 'age_max']
#grouped_multiple = grouped_multiple.reset_index()
grouped_freq = grouped_freq.reset_index().drop(columns=['text','datetime'])
grouped_freq.columns = ['twitter_handle', 'season', 'num_tweets','num_mentions']
grouped_freq.head(5)

,twitter_handle,season,num_tweets,num_mentions
0,1JOLOLO,2010-2011,91,45
1,1JOLOLO,2011-2012,372,187
2,1JOLOLO,2012-2013,842,680
3,1JOLOLO,2013-2014,636,517
4,1JOLOLO,2014-2015,385,275


In [9]:
texts = pd.read_csv('data/player_text_by_season.csv')
len(texts) == len(grouped_freq)

True

In [11]:
texts.sample(5)

,year,twitter_handle,cleaned_text
2442,2018-2019,B_Ingram13,u know a wise man once said nun at all lol ain...
1492,2019-2020,kylekuzma,thanks to my good friend . for the new literat...
684,2019-2020,french_savage,: kendrick perkins first said lebron was the m...
2611,2020-2021,D_Daniels2,: mbb season highlights: deandre daniels () - ...
816,2014-2015,Jwilliams20,: hate is only a form love that hasn't found a...


In [14]:
texts['text_length'] = [len(str(text).split()) for text in texts['cleaned_text']]
texts.sample(5)

,year,twitter_handle,cleaned_text,text_length
1877,2017-2018,mitchmcgary,: jimbo fisher leaving fsu to go to a&amp;m wo...,108
915,2017-2018,rudygobert27,au suivant rt : a rejoint l'quipe hier le est ...,1815
2664,2017-2018,Dsabonis11,: what a nail-biter it was but the third place...,685
989,2014-2015,RODIONS1,: drosme... rt : savage. mjas rt : minjoni + s...,921
2564,2015-2016,TyWallace661,": . averaged points, assists and rebounds duri...",1454


In [17]:
freq = pd.merge(texts, grouped_freq,  how='left', left_on=['twitter_handle','year'], right_on = ['twitter_handle','season'])
freq = freq.drop(columns=['cleaned_text', 'year'])
freq.sample(5)

,twitter_handle,text_length,season,num_tweets,num_mentions
1510,H55ndiaye,311,2014-2015,21,1
1052,StephensonLance,556,2014-2015,59,34
2759,AndreDrummond,6255,2018-2019,547,436
188,rodneyhood,492,2017-2018,56,23
2852,TWroten_LOE,2619,2017-2018,271,106


In [18]:
freq.to_csv('data/frequency_scores.csv',index=False)